In [1]:
import xport
import numpy as np
import pandas as pd
import os
import json
import pyperclip
import json
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import textwrap


from os.path import isfile, join
# import networkx as nx

#https://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_colwidth', 0)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
def describe(df):
    return pd.concat([df.describe().T,
                      df.sum().rename('sum'),
                      df.mad().rename('mad'),
                      df.skew().rename('skew'),
                      df.kurt().rename('kurt'),
                     ], axis=1).T

In [3]:
def concat_columns(data_df, columns_keep, columns_concat, column_rename, column_concat):
    
    concat_df = pd.DataFrame(columns= columns_keep + [column_rename])
    
    concat_df_list = []
    
    for col in columns_concat:
        df = data_df[columns_keep + [col]].rename(columns={col: column_rename})
        df[column_concat] = col
        concat_df_list.append(df)
    
    return pd.concat(concat_df_list).reset_index(drop=True)

In [4]:
SMALL_SIZE = 14
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

matplotlib.rcParams['font.serif'] = "Times New Roman"
matplotlib.rcParams['font.family'] = "serif"
sns.set_style({'font.family':'serif', 'font.serif':'Times New Roman'})

In [5]:
import random

def get_random_color(pastel_factor = 0.5):
    return [(x+pastel_factor)/(1.0+pastel_factor) for x in [random.uniform(0,1.0) for i in [1,2,3]]]

def color_distance(c1,c2):
    return sum([abs(x[0]-x[1]) for x in zip(c1,c2)])

def generate_new_color(existing_colors,pastel_factor = 0.5):
    max_distance = None
    best_color = None
    for i in range(0,100):
        color = get_random_color(pastel_factor = pastel_factor)
        if not existing_colors:
            return color
        best_distance = min([color_distance(color,c) for c in existing_colors])
        if not max_distance or best_distance > max_distance:
            max_distance = best_distance
            best_color = color
    return best_color

def gen_rand_colors(n):
    colors = []

    for i in range(0, n):
        colors.append(generate_new_color(colors))
        
    return colors

# Important Functional Notes



# Load Data


In [6]:
# 57341000    'Cereal (Post Shredded Wheat''n Bran)'

In [7]:
excel_mess={
    'mess': [
        '4:00','6:00','8:00','10:00','12:00','14:00','16:00','18:00','18:01','18:02','18:03','20:04','16:01','18:04','20:01','22:01'
    ],
    'correct': [
        '4:0', '6:0', '8:0', '10:0', '12:0', '14:0', '16:0', '18:0', '18:1', '18:2', '18:3', '20:4', '16:1', '18:4', '20:1', '22:1'
    ]
}

excel_mess_dict = {excel_mess['mess'][i]: excel_mess['correct'][i] for i in range(len(excel_mess['mess']))} 

excel_mess_dict

{'4:00': '4:0',
 '6:00': '6:0',
 '8:00': '8:0',
 '10:00': '10:0',
 '12:00': '12:0',
 '14:00': '14:0',
 '16:00': '16:0',
 '18:00': '18:0',
 '18:01': '18:1',
 '18:02': '18:2',
 '18:03': '18:3',
 '20:04': '20:4',
 '16:01': '16:1',
 '18:04': '18:4',
 '20:01': '20:1',
 '22:01': '22:1'}

In [8]:
def load_data_main():
    global fndds_cons_df
    
    fndds_cons_df = pd.read_csv('FNDDS_2015_Weighted_Consumption_WWEIA_nuts.csv', index_col=0)

    # fndds_FPro_df = pd.read_csv('FNDDS_2015_classified_with_58_nuts.csv')
    # fndds_FPro_df[['Food code', 'FPS', 'ens_FPS', 'ens_min_FPS']],

    fndds_FPro_df = pd.read_csv('FNDDS201520165foldsSMOTE_62Panel.csv')

    fndds_cons_df = pd.merge(
        fndds_FPro_df[['Food code', 'FPS', 'min_FPS', 'std_FPS']],
        fndds_cons_df,
        on='Food code', how='left')

    del fndds_FPro_df

    pass

load_data_main()
len(fndds_cons_df)

C:\anaconda3\envs\foodome\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


8690

In [9]:
# THIS MUST ALWAYS BE FALSE...
'4:00' in fndds_cons_df.columns

False

# Filter Data

In [10]:
cols_messed_name = {
    '4:00': '4:0',
    '6:00': '6:0',
    '8:00': '8:0',
    '10:00': '10:0',
    '12:00': '12:0',
    '14:00': '14:0',
    '16:00': '16:0',
    '16:01': '16:1',
    '18:00': '18:0',
    '18:01': '18:1',
    '18:02': '18:2',
    '18:03': '18:3',
    '18:04': '18:4',
    '20:01': '20:1',
    '20:04': '20:4',
    '22:01': '22:1'
}

In [11]:
'''
WHEAT BRAN... it is from 2011
'''

ONLY_KEEP_TARGETED_YEAR = False

TARGETED_YEAR = ['2009', '2011'][1]

if ONLY_KEEP_TARGETED_YEAR is True:
    nut_panel = '99'
    path_foods_fndds_targetted = f'RFFNDDScleanedwithsalt_additionallab_PREDICTION99_5folds_SMOTE.csv'
else:
    path_foods_fndds_targetted = f'FNDDS_{TARGETED_YEAR}_classified_with_62_nuts.csv'
    nut_panel = '62 but Not Needed!'

foods_fndds_targetted_df = (
    pd.read_csv(path_foods_fndds_targetted)
    .rename(columns={'ens_FPS': 'FPS', 'ens_std_FPS': 'std_FPS', 'ens_min_FPS': 'min_FPS', 'Food_code': 'Food code',
                    'Main_food_description': 'Main food description'})
    .rename(columns=cols_messed_name)
    .drop(columns=['Unnamed: 0'])
)
foods_fndds_targetted_df.head()

# foods_fndds_targetted_df = foods_fndds_targetted_df[foods_fndds_2011_df['Food code'].isin([
#     57601100, # Wheat bran, unprocessed
#     57604100, # Whole wheat, cracked
# ])]

print(path_foods_fndds_targetted)

len(foods_fndds_targetted_df)

FNDDS_2011_classified_with_62_nuts.csv


7618

In [12]:
data_source_name = ''

if ONLY_KEEP_TARGETED_YEAR is True:
    fndds_cons_df = foods_fndds_targetted_df    
    data_source_name = 'FNDDS' + TARGETED_YEAR + 'P' + nut_panel
else:
    load_data_main()

    fndds_cons_df = (
        pd.concat([fndds_cons_df, foods_fndds_targetted_df])
        .drop_duplicates('Food code', keep='first')
        .reset_index(drop=True)
    #     .value_counts('Food code')
    )
    
    data_source_name = 'FNDDS2015+' + TARGETED_YEAR
    pass

print('ONLY_KEEP_TARGETED_YEAR:', ONLY_KEEP_TARGETED_YEAR)
print('data_source_name:', data_source_name)
len(fndds_cons_df)

ONLY_KEEP_TARGETED_YEAR: False
data_source_name: FNDDS2015+2011


9616

In [13]:
# load_data_main()
set(fndds_cons_df.columns) - set(foods_fndds_targetted_df.columns)

# set(foods_fndds_targetted_df.columns) - set(fndds_cons_df.columns)
# list(foods_fndds_targetted_df.columns)

{'Calories',
 'FPS',
 'Grams',
 'WWEIA Category description',
 'WWEIA Category number',
 'cat_digit_1',
 'cat_digit_2',
 'cat_digit_3',
 'cat_digit_4',
 'cat_digit_5',
 'min_FPS',
 'n',
 'se.Calories',
 'se.Grams',
 'se.n',
 'std_FPS',
 'year_WWEIA_category'}

In [14]:
case_studies = [
    {
        'name': 'POST four cereals',
        'type': 'general',
        'food codes':[
            57237300, # Cereal (Post Honey Bunches of Oats with Almonds)
            57230000, # Cereal (Post Grape-Nuts)

            57417000, # Cereal (Post Shredded Wheat)
            57341000, # Cereal (Post Shredded Wheat'n Bran)
        ],
        'comparison base': 57341000, # Cereal (Post Shredded Wheat'n Bran)
    },
    {
        'name': 'POST WheatBran vs ShreddedWheat',
        'type': 'general',
        'food codes':[
            57417000, # Cereal (Post Shredded Wheat)
            57341000, # Cereal (Post Shredded Wheat'n Bran)
        ]
    },
    {
        'name': 'POST four cereals AND UnprocessedWheatBran',
        'type': 'general',
        'food codes':[
            57237300, #Cereal (Post Honey Bunches of Oats with Almonds)
            57230000, # Cereal (Post Grape-Nuts)

            57417000, # Cereal (Post Shredded Wheat)
            57341000, # Cereal (Post Shredded Wheat'n Bran)
            57601100, # Wheat bran, unprocessed
        ]
    },
    {
        'name': 'compare POST four cereals vs UnprocessedWheatBran',
        'type': 'general',
        'food codes':[
            57237300, #Cereal (Post Honey Bunches of Oats with Almonds)
            57230000, # Cereal (Post Grape-Nuts)

            57417000, # Cereal (Post Shredded Wheat)
            57341000, # Cereal (Post Shredded Wheat'n Bran)
            
            57601100, # Wheat bran, unprocessed
        ],
        'comparison base': 57601100, # Wheat bran, unprocessed
    },
    {
        'name': 'POSTS Cereals vs Ingred UnprocessedWheatBran',
        'type': 'general',
        'food codes':[
            57417000, # Cereal (Post Shredded Wheat)
            57341000, # Cereal (Post Shredded Wheat'n Bran)
            
            57601100, # Wheat bran, unprocessed
        ],
        'comparison base': 57601100, # Wheat bran, unprocessed
        #     57601100, # Wheat bran, unprocessed
        #     57604100, # Whole wheat, cracked
    },
    {
        'name': 'Lasagna meat and canned',
        'type': 'general',
        'food codes':[
            58130011, # Lasagna with meat
            58130013, # Lasagna with meat, canned
        ]
    },
    {
        'name': 'compare All Lasagna vs HomeRecipe',
        'type': 'home vs commercial',
        'food codes':[
            58130011, # Lasagna with meat
            58130013, # Lasagna with meat, canned
            58130014, # Lasagna with meat, from restaurant
            
            58130015, # Lasagna with meat, home recipe
        ],
        'comparison base': 58130015 # Lasagna with meat, home recipe
    },
    {
        'name': 'Lasagna ALL FOUR',
        'type': 'general',
        'food codes':[
            58130015, # Lasagna with meat, home recipe
            58130014, # Lasagna with meat, from restaurant

            58130011, # Lasagna with meat
            58130013, # Lasagna with meat, canned
        ]
    },
    {
        'name': 'Salsa home vs restaurant',
        'type': 'home vs commercial',
        'food codes':[
            74402200,  #Salsa, red, homemade
            74402150 # Salsa, red, commercially-prepared
        ],
        'comparison base': 74402200,  #Salsa, red, homemade
    },
    
    {
        'name': 'BeefNoodleSoup home vs restaurant',
        'type': 'home vs commercial',
        'food codes':[
            28310320, # Beef noodle soup, Puerto Rican style
            58402010, # Beef noodle soup, canned or ready-to-serve
            58402100, # Beef noodle soup, home recipe
        ],
        'comparison base': 58402100,  #Salsa, red, homemade
    },
]

def get_case_study(name):
    for case in case_studies:
        if case['name'] == name:
            return case
        
    return None

case_study = [c['name'] for c in case_studies][-1]

case_study = 'POST four cereals AND UnprocessedWheatBran'

case_study = get_case_study(case_study)

# load_data_main()


foods_df = fndds_cons_df[fndds_cons_df['Food code'].isin(case_study['food codes'])]

print(case_study['name'], '| TYPE:', case_study['type'])
foods_df

POST four cereals AND UnprocessedWheatBran | TYPE: general


,Food code,FPS,min_FPS,std_FPS,Main food description,year,cat_digit_1,cat_digit_2,cat_digit_3,cat_digit_4,cat_digit_5,WWEIA Category number,WWEIA Category description,year_WWEIA_category,n,se.n,Grams,Calories,se.Grams,se.Calories,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,Retinol,"Carotene, beta","Carotene, alpha",Vitamin E (alpha-tocopherol),Vitamin D (D2 + D3),"Cryptoxanthin, beta",Lycopene,Lutein + zeaxanthin,Vitamin C,Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, total",Vitamin B-12,"Choline, total",Vitamin K (phylloquinone),Folic acid,"Folate, food","Vitamin E, added","Vitamin B-12, added",Cholesterol,"Fatty acids, total saturated",4:0,6:0,8:0,10:0,12:0,14:0,16:0,18:0,18:1,18:2,18:3,20:4,22:6 n-3,16:1,18:4,20:1,20:5 n-3,22:1,22:5 n-3,"Fatty acids, total monounsaturated","Fatty acids, total polyunsaturated",class,p1,p2,p3,p4,Total Vitamin A,NOVA class
4565,57230000,0.960333,0.953889,0.005969,Cereal (Post Grape-Nuts),2015,Grain Products,"Cereals, not cooked or NS as to cooked",NaN,NaN,NaN,4604.0,"Ready-to-eat cereal, lower sugar (=<21.2g/100g)",2015.0,1.656857e+06,862321.482196,1.363823e+08,4.921889e+08,6.599260e+07,2.380722e+08,11.22,1.81,80.49,0.0,3.50,0.0,0.0,8.86,13.0,0.032,0.02800,0.124,0.467,0.400,0.465,0.00207,0.000340,0.000009,0.000000,0.000002,0.000000,0.00062,0.000000,0.0,0.0,0.000166,0.0001,0.000650,0.000120,0.008620,0.000860,0.000345,0.000000,0.0319,0.000002,0.000326,0.000019,0.0,0.000000,0.0,0.340,0.000,0.0,0.000,0.0,0.002,0.004,0.309,0.014,0.189,0.720,0.061,0.000,0.0,0.003,0.0,0.004,0.0,0.000,0.0,0.210,0.780,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4571,57237300,0.999889,0.999444,0.000248,Cereal (Post Honey Bunches of Oats with Almonds),2015,Grain Products,"Cereals, not cooked or NS as to cooked",NaN,NaN,NaN,4604.0,"Ready-to-eat cereal, lower sugar (=<21.2g/100g)",2015.0,4.198847e+06,938582.034788,2.295714e+08,9.396220e+08,5.478322e+07,2.242224e+08,7.70,7.30,79.60,0.0,2.90,0.0,0.0,20.00,5.5,0.035,0.03380,0.061,0.172,0.211,0.417,0.00090,0.000200,0.000012,0.000675,0.000043,0.000028,0.00186,0.000003,0.0,0.0,0.000630,0.0000,0.001200,0.001300,0.015600,0.001600,0.000625,0.000005,0.0214,0.000003,0.000606,0.000019,0.0,0.000005,0.0,0.800,0.001,0.0,0.000,0.0,0.001,0.003,0.606,0.133,3.956,1.974,0.126,0.000,0.0,0.016,0.0,0.024,0.0,0.003,0.0,4.000,2.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4645,57341000,0.565796,0.330741,0.131864,Cereal (Post Shredded Wheat'n Bran),2015,Grain Products,"Cereals, not cooked or NS as to cooked",NaN,NaN,NaN,4604.0,"Ready-to-eat cereal, lower sugar (=<21.2g/100g)",2015.0,5.911034e+04,37205.588817,3.717913e+06,1.258864e+07,2.219281e+06,7.513263e+06,10.93,2.06,80.65,0.0,3.14,0.0,0.0,0.75,14.7,0.044,0.00387,0.174,0.418,0.391,0.000,0.00313,0.000000,0.000008,0.000000,0.000000,0.000000,0.00109,0.000000,0.0,0.0,0.000236,0.0000,0.000200,0.000080,0.005200,0.000080,0.000030,0.000000,0.0231,0.000002,0.000000,0.000030,0.0,0.000000,0.0,0.430,0.000,0.0,0.000,0.0,0.000,0.004,0.405,0.021,0.316,1.166,0.054,0.000,0.0,0.014,0.0,0.000,0.0,0.000,0.0,0.330,1.220,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4672,57417000,0.568500,0.212222,0.200677,Cereal (Post Shredded Wheat),2015,Grain Products,"Cereals, not cooked or NS as to cooked",NaN,NaN,NaN,4604.0,"Ready-to-eat cereal, lower sugar (=<21.2g/100g)",2015.0,1.333248e+06,330919.115972,5.455256e+07,1.876944e+08,1.713816e+07,5.892278e+07,11.58,2.06,80.18,0.0,4.55,0.0,0.0,0.92,12.4,0.057,0.00256,0.132,0.380,0.384,0.002,0.00297,0.000357,0.000004,0.000000,0.000000,0.000000,0.00032,0.000000,0.0,0.0,0.000088,0.0000,0.000285,0.000090,0.005950,0.000155,0.000042,0.000000,0.0231,0.000001,0.000000,0.000042,0.0,0.000000,0.0,0.405,0.000,0.0,0.012,0.0,0.000,0.002,0.372,0.019,0.278,1.188,0.070,0.002,0.0,0.008,0.0,0.008,0.0,0.000,0.0,0.295,1.260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4675,57601100,0.068167,0.041389,0.049966,"Wheat bran, unprocessed",2015,Grain Products,"Cereals, not cooked or NS as

In [15]:
[c for c in foods_df.columns if '4:' in c]

['4:0', '14:0']

In [16]:
def prep_data_for_plot(foods_df, col_nuts):
    print('case_study:', case_study)
    
    foods_df = foods_df.copy()
    
    if bool((foods_df[col_nuts] == -20).any(axis=1).any()) is True:
        print('Already data is in log scale... going to convert back for showing actual values in hover')
        
        foods_df.loc[:, col_nuts] = foods_df.loc[:, col_nuts].replace(-20, np.nan).apply(np.exp).replace(np.nan, 0)
    
    #'THIS IS NEEDED FOR PLOTTING!:'
    
#     col_nuts = list(foods_df.loc[:, 'Protein':'Fatty acids, total polyunsaturated'].columns)
    
    if col_nuts[0] == 'Protein':
        col_nuts.reverse()
#     col_nuts.reverse()

    col_nut_code_dict = {col: i for i, col in enumerate(col_nuts)}
    rows = []

    def un_pivot(r):
        d = {
            'Food code': r['Food code'],
            'FPS': r['FPS'],
    #         'ens_FPS': r['ens_FPS'],
    #         'ens_min_FPS': r['ens_min_FPS'],
            'std_FPS': r['std_FPS'],
            'min_FPS': r['min_FPS'],
            'Main food description': r['Main food description']
        }

        for col in col_nuts:
            
            
            d_copy = d.copy()
            d_copy['nut name'] = col
            d_copy['nut code'] = col_nut_code_dict[col]
            d_copy['nut value'] = r[col]
            d_copy['hover'] = '{}: {}<br />{}<br />Food code:{}'.format(
                col, r[col], r['Main food description'], r['Food code'])
            
            
            if col == 'Calcium':
                d_copy['nut value'] = r[col]
            
            rows.append(d_copy)
            pass

        return r

    q=foods_df.apply(un_pivot, axis=1)

    plot_df = pd.DataFrame(rows).reset_index(drop=True)

    # transform
    plot_df['nut value grams'] = plot_df['nut value']
    
    plot_df['nut value'] = (
        plot_df['nut value']
        .apply(np.log)
        .replace([-np.inf, np.inf], -20)
        .replace(np.nan, -20)
    )
    
    return plot_df

# Plot Nutrients

In [17]:
[c['name'] for c in case_studies]

['POST four cereals',
 'POST WheatBran vs ShreddedWheat',
 'POST four cereals AND UnprocessedWheatBran',
 'compare POST four cereals vs UnprocessedWheatBran',
 'POSTS Cereals vs Ingred UnprocessedWheatBran',
 'Lasagna meat and canned',
 'compare All Lasagna vs HomeRecipe',
 'Lasagna ALL FOUR',
 'Salsa home vs restaurant',
 'BeefNoodleSoup home vs restaurant']

In [21]:
# case_study = 'POSTS Cereals vs Ingred UnprocessedWheatBran'
case_study = ['POST four cereals AND UnprocessedWheatBran', 'compare POST four cereals vs UnprocessedWheatBran'][1]
# case_study = ['Lasagna ALL FOUR', 'Lasagna meat and canned', 'compare All Lasagna vs HomeRecipe'][1]

compare_nuts = True

case_study = get_case_study(case_study)

print(case_study)

if 'Total isoflavones' in foods_df.columns:
    col_nuts = list(foods_df.loc[:, 'Protein':'Total isoflavones'].columns)
else:
    col_nuts = list(foods_df.loc[:, 'Protein':'Fatty acids, total polyunsaturated'].columns)

# load_data_main()
foods_df = (
    fndds_cons_df[fndds_cons_df['Food code'].isin(case_study['food codes'])]
#     .sort_values(by='FPS', ascending=False) # IF YOU SORT, WONT GET SAME ORDER AS SI
)

print('num nuts:', len(col_nuts))

foods_df[col_nuts]
# foods_df

{'name': 'compare POST four cereals vs UnprocessedWheatBran', 'type': 'general', 'food codes': [57237300, 57230000, 57417000, 57341000, 57601100], 'comparison base': 57601100}
num nuts: 62


,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,Retinol,"Carotene, beta","Carotene, alpha",Vitamin E (alpha-tocopherol),Vitamin D (D2 + D3),"Cryptoxanthin, beta",Lycopene,Lutein + zeaxanthin,Vitamin C,Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, total",Vitamin B-12,"Choline, total",Vitamin K (phylloquinone),Folic acid,"Folate, food","Vitamin E, added","Vitamin B-12, added",Cholesterol,"Fatty acids, total saturated",4:0,6:0,8:0,10:0,12:0,14:0,16:0,18:0,18:1,18:2,18:3,20:4,22:6 n-3,16:1,18:4,20:1,20:5 n-3,22:1,22:5 n-3,"Fatty acids, total monounsaturated","Fatty acids, total polyunsaturated"
4565,11.22,1.81,80.49,0.0,3.50,0.0,0.0,8.86,13.0,0.032,0.02800,0.124,0.467,0.400,0.465,0.00207,0.000340,0.000009,0.000000,0.000002,0.000000,0.00062,0.000000,0.0,0.0,0.000166,0.0001,0.000650,0.000120,0.008620,0.000860,0.000345,0.000000,0.0319,0.000002,0.000326,0.000019,0.0,0.000000,0.0,0.340,0.000,0.0,0.000,0.0,0.002,0.004,0.309,0.014,0.189,0.720,0.061,0.000,0.0,0.003,0.0,0.004,0.0,0.000,0.0,0.210,0.780
4571,7.70,7.30,79.60,0.0,2.90,0.0,0.0,20.00,5.5,0.035,0.03380,0.061,0.172,0.211,0.417,0.00090,0.000200,0.000012,0.000675,0.000043,0.000028,0.00186,0.000003,0.0,0.0,0.000630,0.0000,0.001200,0.001300,0.015600,0.001600,0.000625,0.000005,0.0214,0.000003,0.000606,0.000019,0.0,0.000005,0.0,0.800,0.001,0.0,0.000,0.0,0.001,0.003,0.606,0.133,3.956,1.974,0.126,0.000,0.0,0.016,0.0,0.024,0.0,0.003,0.0,4.000,2.100
4645,10.93,2.06,80.65,0.0,3.14,0.0,0.0,0.75,14.7,0.044,0.00387,0.174,0.418,0.391,0.000,0.00313,0.000000,0.000008,0.000000,0.000000,0.000000,0.00109,0.000000,0.0,0.0,0.000236,0.0000,0.000200,0.000080,0.005200,0.000080,0.000030,0.000000,0.0231,0.000002,0.000000,0.000030,0.0,0.000000,0.0,0.430,0.000,0.0,0.000,0.0,0.000,0.004,0.405,0.021,0.316,1.166,0.054,0.000,0.0,0.014,0.0,0.000,0.0,0.000,0.0,0.330,1.220
4672,11.58,2.06,80.18,0.0,4.55,0.0,0.0,0.92,12.4,0.057,0.00256,0.132,0.380,0.384,0.002,0.00297,0.000357,0.000004,0.000000,0.000000,0.000000,0.00032,0.000000,0.0,0.0,0.000088,0.0000,0.000285,0.000090,0.005950,0.000155,0.000042,0.000000,0.0231,0.000001,0.000000,0.000042,0.0,0.000000,0.0,0.405,0.000,0.0,0.012,0.0,0.000,0.002,0.372,0.019,0.278,1.188,0.070,0.002,0.0,0.008,0.0,0.008,0.0,0.000,0.0,0.295,1.260
4675,15.55,4.25,64.51,0.0,9.89,0.0,0.0,0.41,42.8,0.073,0.01057,0.611,1.013,1.182,0.002,0.00727,0.000998,0.000078,0.000000,0.000006,0.000000,0.00149,0.000000,0.0,0.0,0.000240,0.0000,0.000523,0.000577,0.013578,0.001303,0.000079,0.000000,0.0744,0.000002,0.000000,0.000079,0.0,0.000000,0.0,0.630,0.000,0.0,0.000,0.0,0.002,0.007,0.556,0.037,0.619,2.039,0.167,0.005,0.0,0.017,0.0,0.000,0.0,0.000,0.0,0.637,2.212


In [22]:
'''
log(items) - log(base)
'''

# foods_cmp_df = foods_df[col_nuts]

if 'comparison base' in case_study and case_study['comparison base'] is not None:
    compare_nuts = True
    foods_cmp_df = None

if compare_nuts:
    mask_compare_to = foods_df['Food code'] != case_study['comparison base']

    foods_cmp_df = (
        foods_df.loc[mask_compare_to, col_nuts] / 
        foods_df.loc[foods_df['Food code'] == case_study['comparison base'], col_nuts].iloc[0]
    )

    foods_cmp_df = foods_cmp_df.replace([np.inf, -np.inf], None)

    nan_cols = [i for i in foods_cmp_df.columns if foods_cmp_df[i].isnull().any()]
    
    print('compare_nuts:', compare_nuts)
    print(len(foods_cmp_df[col_nuts]))
    pass

compare_nuts: True
4


In [23]:
'''Assign log of devisions for targeted items'''

if compare_nuts:
    foods_df.loc[mask_compare_to, col_nuts] = foods_cmp_df.loc[:, col_nuts]
    
    foods_df = foods_df[foods_df['Food code'] != case_study['comparison base']]
    
foods_df[col_nuts]

C:\anaconda3\envs\foodome\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,Retinol,"Carotene, beta","Carotene, alpha",Vitamin E (alpha-tocopherol),Vitamin D (D2 + D3),"Cryptoxanthin, beta",Lycopene,Lutein + zeaxanthin,Vitamin C,Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, total",Vitamin B-12,"Choline, total",Vitamin K (phylloquinone),Folic acid,"Folate, food","Vitamin E, added","Vitamin B-12, added",Cholesterol,"Fatty acids, total saturated",4:0,6:0,8:0,10:0,12:0,14:0,16:0,18:0,18:1,18:2,18:3,20:4,22:6 n-3,16:1,18:4,20:1,20:5 n-3,22:1,22:5 n-3,"Fatty acids, total monounsaturated","Fatty acids, total polyunsaturated"
4565,0.721543,0.425882,1.247714,NaN,0.353893,NaN,NaN,21.609756,0.303738,0.438356,2.649007,0.202946,0.461007,0.338409,232.5,0.284732,0.340681,0.117268,NaN,0.333333,NaN,0.416107,NaN,NaN,NaN,0.691667,inf,1.242830,0.207972,0.634850,0.660015,4.367089,NaN,0.428763,1.052632,inf,0.240506,NaN,NaN,NaN,0.539683,NaN,NaN,NaN,NaN,1.0,0.571429,0.555755,0.378378,0.305331,0.353114,0.365269,0.0,NaN,0.176471,NaN,inf,NaN,NaN,NaN,0.329670,0.352622
4571,0.495177,1.717647,1.233917,NaN,0.293225,NaN,NaN,48.780488,0.128505,0.479452,3.197729,0.099836,0.169793,0.178511,208.5,0.123796,0.200401,0.154639,NaN,7.166667,NaN,1.248322,NaN,NaN,NaN,2.625000,NaN,2.294455,2.253033,1.148917,1.227936,7.911392,NaN,0.287634,1.631579,inf,0.240506,NaN,NaN,NaN,1.269841,NaN,NaN,NaN,NaN,0.5,0.428571,1.089928,3.594595,6.390953,0.968122,0.754491,0.0,NaN,0.941176,NaN,inf,NaN,NaN,NaN,6.279435,0.949367
4645,0.702894,0.484706,1.250194,NaN,0.317492,NaN,NaN,1.829268,0.343458,0.602740,0.366131,0.284779,0.412636,0.330795,0.0,0.430536,0.000000,0.105670,NaN,0.000000,NaN,0.731544,NaN,NaN,NaN,0.983333,NaN,0.382409,0.138648,0.382972,0.061397,0.379747,NaN,0.310484,1.052632,NaN,0.379747,NaN,NaN,NaN,0.682540,NaN,NaN,NaN,NaN,0.0,0.571429,0.728417,0.567568,0.510501,0.571849,0.323353,0.0,NaN,0.823529,NaN,NaN,NaN,NaN,NaN,0.518053,0.551537
4672,0.744695,0.484706,1.242908,NaN,0.460061,NaN,NaN,2.243902,0.289720,0.780822,0.242195,0.216039,0.375123,0.324873,1.0,0.408528,0.357715,0.056701,NaN,0.000000,NaN,0.214765,NaN,NaN,NaN,0.366667,NaN,0.544933,0.155979,0.438209,0.118956,0.531646,NaN,0.310484,0.736842,NaN,0.531646,NaN,NaN,NaN,0.642857,NaN,NaN,NaN,NaN,0.0,0.285714,0.669065,0.513514,0.449111,0.582639,0.419162,0.4,NaN,0.470588,NaN,NaN,NaN,NaN,NaN,0.463108,0.569620


In [24]:
'''Only plot items that we have comparision (not divided by zero)'''
if compare_nuts:
#     col_nuts = [c for c in col_nuts if c not in nan_cols]
    col_nuts = col_nuts
    pass

len(col_nuts)

62

In [25]:
np.log(-2.120263536)

<ipython-input-25-86c13a1633f2>:1: RuntimeWarning: invalid value encountered in log
  np.log(-2.120263536)


nan

In [26]:
foods_df.loc[:, col_nuts].replace(-20, np.nan).apply(np.exp).replace(np.nan, 0)
(foods_df[col_nuts] == -20).any(axis=1).any()
bool((foods_df[col_nuts] == -20).any(axis=1).any()) is True

False

In [27]:
'''Prepare data for the plot'''
plot_df = prep_data_for_plot(foods_df=foods_df, col_nuts=col_nuts)

print(case_study['name'], '| TYPE:', case_study['type'])
# foods_df
plot_df

case_study: {'name': 'compare POST four cereals vs UnprocessedWheatBran', 'type': 'general', 'food codes': [57237300, 57230000, 57417000, 57341000, 57601100], 'comparison base': 57601100}
compare POST four cereals vs UnprocessedWheatBran | TYPE: general


,Food code,FPS,std_FPS,min_FPS,Main food description,nut name,nut code,nut value,hover,nut value grams
0,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),"Fatty acids, total polyunsaturated",0,-1.042358,"Fatty acids, total polyunsaturated: 0.35262206148282094<br />Cereal (Post Grape-Nuts)<br />Food code:57230000",0.352622
1,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),"Fatty acids, total monounsaturated",1,-1.109662,"Fatty acids, total monounsaturated: 0.32967032967032966<br />Cereal (Post Grape-Nuts)<br />Food code:57230000",0.329670
2,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),22:5 n-3,2,-20.000000,22:5 n-3: nan<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,NaN
3,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),22:1,3,-20.000000,22:1: nan<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,NaN
4,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),20:5 n-3,4,-20.000000,20:5 n-3: nan<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,NaN
5,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),20:1,5,-20.000000,20:1: inf<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,inf
6,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),18:4,6,-20.000000,18:4: nan<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,NaN
7,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),16:1,7,-1.734601,16:1: 0.1764705882352941<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,0.176471
8,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),22:6 n-3,8,-20.000000,22:6 n-3: nan<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,NaN
9,57230000,0.960333,0.005969,0.953889,Cereal (Post Grape-Nuts),20:4,9,-20.000000,20:4: 0.0<br />Cereal (Post Grape-Nuts)<br />Food code:57230000,0.000000


In [28]:
# file_save = ['Ready-to-eat cereal.html', 'Nutrition bars.html', 'wheat_ingredients_from_FNDDS_2011.html',
#             'Lasagna.html'][3]
if compare_nuts:
#     xaxis_title = 'log(cereal) - log(UnprocessedWheatBran)<br>vs.<br>log(x) for UnprocessedWheatBran'
#     xaxis_title = 'log(cereal) - log(UnprocessedWheatBran)'
    xaxis_title = r'$log(x_{cereal} / x_{WheatBran})$'
else:
    xaxis_title = r'$log(x)$'
    pass

if False:
    plot_title = None
else:
    plot_title = '{} | DataSource: {}'.format(case_study['name'].title(), data_source_name)

font_size = 16
width=[1200, 900][-1]
height=1200

if len(col_nuts) > 62:
    height = 1400
    
file_save = (case_study['name'] + '_source_' + data_source_name).replace(' ', '_') + '.html'

FPro_type = ['FPS', 'ens_FPS' ,'ens_min_FPS'][2]

layout = go.Layout(
  margin=go.layout.Margin(
        l=0, #left margin
        r=0, #right margin
        b=0, #bottom margin
        t=0 if plot_title is None else 40  #top margin
    )
)

fig = go.Figure(layout=layout)

# colors = gen_rand_colors(len(plot_df['Food code'].unique()))

if len(foods_df) == 2:
    colors = ['#e6194B', '#469990', '#ffe119', '#911eb4']
    marker_symbols = ['circle', 'circle', 'circle', 'circle']
else:
    colors = ['#911eb4', '#e6194B', '#ffe119', '#469990', 'black']
    marker_symbols = ['cross', 'star-triangle-up', 'circle', 'diamond', 'cross']
    pass

markers_opacity = 0.75
marker_size = [6, 8][1]
max_x = -20

for i, food_code in enumerate(plot_df['Food code'].unique()):
    tmp_df = plot_df[plot_df['Food code'] == food_code]
    
    if np.ceil(max(tmp_df['nut value'])) > max_x:
        max_x = np.ceil(max(tmp_df['nut value']))
        
    food_desc = tmp_df['Main food description'].unique()[0]
    
    FPro = tmp_df['FPS'].unique()[0]
#     FPro_ens_avg = tmp_df['ens_FPS'].unique()[0]
#     FPro_ens_min = tmp_df['ens_min_FPS'].unique()[0]
    FPro_ens_min = tmp_df['min_FPS'].unique()[0]
    FPro_ens_std = tmp_df['std_FPS'].unique()[0]
    
    food_desc = '<br />'.join(textwrap.wrap(food_desc, 15))

#     food_desc = '<br />'.join(textwrap.wrap(food_desc, 30))

#     food_desc = food_desc + '<br />FPro:({:.3f}, min:{:.3f}, std:{:.3f})'.format(FPro, FPro_ens_min, FPro_ens_std)
    food_desc = food_desc + '<br />FPro: {:.4f}'.format(FPro, FPro_ens_min, FPro_ens_std)
    food_desc = food_desc + '<br />'
    

    marker = dict(
        size=marker_size, symbol = 'circle', opacity=markers_opacity, #line=dict(width=1,color='DarkSlateGrey')
        color=colors[i], 
        # color='rgb({}, {}, {})'.format(colors[i][0]*255, colors[i][1]*255, colors[i][2]*255)
    )
    
    line=dict(
        width=1, dash='dash',
        color='rgb({}, {}, {})'.format(colors[i][0]*255, colors[i][1]*255, colors[i][2]*255)
        # color='rgba(255, 0, 0, 0.5)' # FOR THIS TO WORK YOU MUST HANDLE ALL COLORS YOURSELF
     )
    
    fig.add_trace(
            go.Scatter(
                x=tmp_df['nut value'],
                y=tmp_df['nut code'], 
                hovertext=tmp_df['hover'], 
#                 line=line,
                marker=marker,
                mode=['markers', 'markers+lines'][0],
                name=food_desc,
                marker_symbol=marker_symbols[i]
        ))
    pass

if compare_nuts:
    fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="black")


# more on gridlines: https://codepen.io/MojtabaSamimi/pen/qBOwjoq?editors=0010
fig.update_layout(
    title = plot_title,
    width=width,
    height=height,
    yaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0, len(col_nuts))),
        ticktext = col_nuts,
        zeroline=False, #adds zeroline!
        gridcolor= "rgb(231, 222, 222)"
    ),
    xaxis= dict(
        title=xaxis_title,
#       "tickcolor": "orange",
#       "tickwidth": 50,
        gridcolor= "rgb(231, 222, 222)",
        gridwidth= 1,
        zeroline=False, #adds zeroline!
#         dtick=1,
        tickvals=np.arange(-20, max_x + 1, 1)
    ),
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font= {
        'family': 'Times New Roman',
        'size': font_size,
        'color': '#000000'
    }
)

# ‘paper_bgcolor’: ‘rgba(0, 0, 0, 0)

fig.update_xaxes(range=[-21, 6], autorange=False)
fig.update_yaxes(range=[-1, len(col_nuts)], autorange=False)
# fig.update_xaxes(autorange=False)

# fig.write_image(file_save + '.pdf', format='pdf', scale=8)
# print(file_save + '.pdf')
fig.write_image(file_save + '.png', format='png', scale=8)
print(file_save + '.png')

fig.write_html(file_save)

pio.show(fig, renderer='browser', validate=True)

compare_POST_four_cereals_vs_UnprocessedWheatBran_source_FNDDS2015+2011.html.png


In [29]:
print(plot_df['Food code'].value_counts())

# plot_df.to_csv('D:/FProXFinal/FigS10/FigS10_A.csv')
# plot_df.to_csv('D:/FProXFinal/FigS10/FigS10_B.csv')

57417000    62
57237300    62
57341000    62
57230000    62
Name: Food code, dtype: int64


In [30]:
raise Exception('DONE!')

Exception: DONE!

In [ ]:
fig.write_html(file_save)
file_save

# Raw Items FPro Plot with Stadard Error!

In [ ]:
raw_foods = (
    fndds_cons_df[
        (fndds_cons_df['Main food description'].str.contains(', raw')) & 
        (fndds_cons_df['cat_digit_1'].isin(['Fruits', 'Vegetables'])) &
        (~fndds_cons_df['Main food description'].str.contains('salad'))
    ]
    .sort_values(by='FPS', ascending=True)
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={'index': 'rank'})
)

# raw_foods

In [ ]:
fig = go.Figure()

if False:
    for cat in raw_foods['cat_digit_1'].unique():
        raw_foods_cat_df = raw_foods[raw_foods['cat_digit_1'] == cat]

        fig.add_trace(
                go.Scatter(
                    x=raw_foods_cat_df['FPS'],
                    y=raw_foods_cat_df['rank'],
    #                 hovertext=tmp_df['hover'], 
    #                 line=line,
    #                 marker=marker,
                    mode=['markers', 'markers+lines'][0],
                    name=cat,
                    marker_symbol='circle',
    #                 error_x=raw_foods_cat_df["std_FPS"], 
    #                 error_y=raw_foods_cat_df["std_FPS"]
            )
        )

        pass


    
fig = px.scatter(
    raw_foods, x="FPS", y="rank", color="cat_digit_1",
    error_x="std_FPS", 
#     error_y="std_FPS", 
    hover_data=['Main food description']
)

# Main food description
fig.update_traces(marker_size=5, marker_symbol='circle')

fig.update_layout(
#     title = 'FPro type={} | {}'.format(FPro_type, file_save),
    width=900,
    height=1500,
    legend_title='',
    yaxis = dict(
        title='',
        tickmode = 'array',
        tickvals = list(range(0,len(raw_foods))),
        range=[-1,len(raw_foods)],
        ticktext = raw_foods['Main food description'],
        zeroline=False,
        gridcolor= "rgb(231, 222, 222)",
    ),
    xaxis= dict(
        title='FPro',
#       "tickcolor": "orange",
#       "tickwidth": 50,
        gridcolor= "rgb(231, 222, 222)",
        gridwidth= 1,
        zeroline=False,
#         dtick=1,
#         tickvals=np.arange(-20, max_x + 1, 1)
    ),
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font= {
        'family': 'Times New Roman',
        'size': 11,
        'color': '#000000'
    }
)

# pio.show(fig, renderer='browser', validate=True)

fig.data[0].error_x.thickness = 1
fig.data[1].error_x.thickness = 1

# fig.show()
file_name = 'raw_foods_FPro_FNDDS_2015_62p.pdf'
fig.write_image(file_name, format='pdf', scale=2)
print(file_name)

- https://jakevdp.github.io/PythonDataScienceHandbook/04.03-errorbars.html

# Compare Nuts

In [ ]:
[c['name'] for c in case_studies]

In [ ]:
case_study = 'POSTS Cereals vs Ingred UnprocessedWheatBran'

case_study = get_case_study(case_study)

# load_data_main()
foods_df = fndds_cons_df[fndds_cons_df['Food code'].isin(case_study['food codes'])]

col_nuts = list(foods_df.loc[:, 'Protein':'Fatty acids, total polyunsaturated'].columns)

plot_df = prep_data_for_plot(foods_df=foods_df, col_nuts=col_nuts)

print(case_study['name'], '| TYPE:', case_study['type'])
foods_df

In [ ]:
def compare_nut_diff(ignore_on_compare_with_zero, verbos_print):
    comparison_base_df = plot_df[plot_df['Food code'] == case_study['comparison base']]

    comp_with_results = {}

    for food_code in plot_df['Food code'].unique():
        if food_code == case_study['comparison base']:
            continue       
        
        comp_with_results[food_code] = []

        comparison_with_df = plot_df[plot_df['Food code'] == food_code]
        
        
        print('-------------comparing with: {} ({})-------------'.format(
            comparison_with_df['Main food description'].values[0],
            comparison_with_df['Food code'].values[0],
        ))

        for nut_code in plot_df['nut code'].unique():
            base_val = comparison_base_df[comparison_base_df['nut code'] == nut_code]['nut value grams'].values[0]

            comp_with_val = comparison_with_df[comparison_with_df['nut code'] == nut_code]['nut value grams'].values[0]

            nut_name = comparison_base_df[comparison_base_df['nut code'] == nut_code]['nut name'].values[0]

            if base_val == comp_with_val == 0:
                perc_diff = 0
            elif base_val == 0 and comp_with_val != 0:
                perc_diff = 1
                print(f'!!Base 0 but comp NotZero ({nut_name})', '| ignore:', ignore_on_compare_with_zero)

                if ignore_on_compare_with_zero:
                    continue
            elif base_val != 0 and comp_with_val == 0:
                perc_diff = 1
                print(f'~~Base NotZero but comp 0 ({nut_name})', '| ignore:', ignore_on_compare_with_zero)

                if ignore_on_compare_with_zero:
                    continue
            else:
                perc_diff = np.abs((base_val - comp_with_val) / base_val)

            comp_with_results[food_code].append(perc_diff)

            if verbos_print:
                print(
                    'nut_code:', nut_code, 
                    '| base val:', base_val,
                    '| comp with val:', comp_with_val,
                    '| perc_diff:', perc_diff
                )
            pass
        
        comp_with_results[food_code] = np.mean(comp_with_results[food_code])

        print('@@ ABS Diff:', comp_with_results[food_code])
        pass
    
    return comp_with_results
# comparison_base_df

print('Case Study:', case_study)
comp_res = compare_nut_diff(ignore_on_compare_with_zero = True, verbos_print = False)

# print(comp_res)

comp_res_df = pd.merge(
    foods_df[['Food code', 'FPS', 'Main food description']],
    pd.DataFrame([(k, v) for k,v in comp_res.items()], columns=['Food code', 'Comp Res']),
    on='Food code', how='left'
).sort_values(by='FPS')

comp_res_df

# Estimation Food Source Restournt vs ETC

In [ ]:
food_source_df = pd.read_csv('D:/Dropbox (CCNR)/Foodome Team Folder/Ravandi, Babak FDB/FoodomeDev/NHANES_2003_2018_FoodSource_Consumed.csv')

food_source_df = food_source_df[
    (food_source_df['Dietary recall status'] == 1) & 
    (food_source_df['year'].isin([2003, 2005]))
].reset_index(drop=True)

# Water removal did not make so much change!
# water_food_codes = [94000000, 94000100, 94100100, 94300100]
# food_source_df = food_source_df[~food_source_df['Food code'].isin(water_food_codes)]

len(food_source_df)
# Just plot percentage of grams consumed in every category within 2013-2016

In [ ]:
sum_grams_consumed = food_source_df['Grams consumed'].sum()

food_grams_perc_df = (
    food_source_df.groupby('Food source')
    .agg({'Grams consumed': np.sum})
    / sum_grams_consumed
)

food_grams_perc_df = (
    food_grams_perc_df
    .sort_values(by='Grams consumed', ascending=False) * 100
).reset_index().rename(columns={'Grams consumed': '% of Grams Consumed'})

food_grams_perc_df

In [ ]:
fig = plt.figure(figsize=(15, 15), dpi=150)

sns.set(style="ticks", font="Times New Roman", font_scale=2)

ax = sns.barplot(
    data=food_grams_perc_df, 
    x='% of Grams Consumed', 
    y='Food source', 
#     hue='status',
#     order=list(gdb_status_count_order_df[column_cat].values)
)

# Manul NOVA with Uncertain FPro

In [ ]:
nova_df = pd.read_excel('FNDDS_food_ingreds_2015_NOVA.xls')
fndds_FPro_df = pd.read_csv('FNDDS201520165foldsSMOTE_62Panel.csv')

nova_df = nova_df.drop_duplicates('dr12ifdcd')

ingreds_df = pd.read_excel('FNDDS_food_ingreds_2015_NOVA.xls')
ingreds_df = ingreds_df[['dr12ifdcd', 'FNDDS_food_description', 'Ingredientcode',
       'FNDDS_ingred_food_description', 'SR_code_t', 'SR_description_t',
       'Recursionlevel', 'frac', 'weight']].rename(columns={'dr12ifdcd': 'Food code'})

In [ ]:
nova_fpro_df = pd.merge(
    fndds_FPro_df[['Food code', 'FPS', 'cat_digit_1', 'cat_digit_2', 'cat_digit_3', 'WWEIACategorydescription']],
    nova_df.rename(columns={'dr12ifdcd': 'Food code', 'FC_nova_group': 'nova_class'})[['Food code', 'FNDDS_food_description', 'nova_class']],
    on='Food code'
).sort_values('FPS')

nova_fpro_df['nova_class'].value_counts()

In [ ]:
# nova_fpro_df[(nova_fpro_df['nova_class'] == 4) & (nova_fpro_df['FPS'] < 0.7)]
(
    nova_fpro_df[
        (nova_fpro_df['nova_class'] == 4) & (nova_fpro_df['FPS'] < 0.7)
#         & (nova_fpro_df['WWEIACategorydescription'].isin(['Chicken, whole pieces']))
    ]
    .sort_values(by='FPS', ascending=True)
#     ['WWEIACategorydescription'].value_counts()
#     .sum()
)

In [ ]:
nova_uncertain = pd.merge(
    nova_fpro_df[(nova_fpro_df['nova_class'] == 4) & (nova_fpro_df['FPS'] < 0.7)].drop(columns='FNDDS_food_description'),
    ingreds_df, on='Food code'
)
nova_uncertain

# EWAS Category Color Plot

In [ ]:
df_summary = pd.read_csv('D:/Dropbox (CCNR)/Foodome Team Folder/Menichetti, Giulia/FoodProcessing/Datasets/00 Final Package/df_summary.csv')

colors_dict_str = df_summary.drop_duplicates('category').set_index('category').sort_index()['color'].to_dict()

colors = {color: eval(rgba_str) for color, rgba_str in colors_dict_str.items()}

In [ ]:
[(tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                    for name, color in colors.items()]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors


if False:
    # colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

    # Sort colors by hue, saturation, value and name.
    by_hsv = sorted(
        (tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                    for name, color in colors.items()
    )
else:
    by_hsv = [(tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                    for name, color in colors.items()]
    pass

sorted_names = [name for hsv, name in by_hsv]
    
n = len(sorted_names)
ncols = 1
nrows = n // ncols
if nrows == 0:
    nrows = 1

fig, ax = plt.subplots(figsize=(12, 10))

# Get height and width
X, Y = fig.get_dpi() * fig.get_size_inches()
h = Y / (nrows + 1)
w = X / ncols

for i, name in enumerate(sorted_names):
    row = i % nrows
    col = i // nrows
    y = Y - (row * h) - h

    xi_line = w * (col + 0.05)
    xf_line = w * (col + 0.25)
    xi_text = w * (col + 0.3)

    ax.text(xi_text, y, name, fontsize=(h * 0.8),
            horizontalalignment='left',
            verticalalignment='center')

    ax.hlines(y + h * 0.1, xi_line, xf_line,
              color=colors[name], linewidth=(h * 0.8))

ax.set_xlim(0, X)
ax.set_ylim(0, Y)
ax.set_axis_off()

fig.subplots_adjust(left=0, right=1,
                    top=1, bottom=0,
                    hspace=0, wspace=0)
plt.show()

# Challenges

- https://plotly.com/python/tick-formatting/
- https://plotly.com/python/marker-style/
- https://plotly.com/python/line-charts/
- line opacity colors https://stackoverflow.com/questions/50488894/plotly-py-change-line-opacity-leave-markers-opaque
- https://stackoverflow.com/questions/470690/how-to-automatically-generate-n-distinct-colors
- https://stackoverflow.com/questions/22408237/named-colors-in-matplotlib